## Demonstration of the User-Items Bias Baseline Recommender Model
In this file, the demonstration of how the user-item bias baseline model recommends news for a user is presented. 
It also includes the evaluation of the recommender model by calculating the metrics *Precision@K and NDCG@K*.

In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils.process_data import user_item_interaction_scores
from parquet_data_reader import ParquetDataReader
from models.baseline import UserItemBiasRecommender


import polars as pl
pl.Config.set_tbl_cols(-1)
import numpy as np
parquet_reader = ParquetDataReader()

### Reading the Data and Preprocessing

In [2]:
train_behavior_df = parquet_reader.read_data("../../data/train/behaviors.parquet")
embeddings_df = parquet_reader.read_data("../../data/document_vector.parquet")
article_df = parquet_reader.read_data("../../data/articles.parquet")
test_behavior_df = parquet_reader.read_data("../../data/validation/behaviors.parquet")
processed_behavior_df = user_item_interaction_scores(train_behavior_df, article=article_df)

### Creates and Fits the Model to the Data

In [3]:
model = UserItemBiasRecommender(processed_behavior_df)
model.fit()

### Recommendations
Produces the recommended article-ids for the provided user id.

It uses the function: *r_hat(u, i) = mu + b_u + b_i*

In [4]:
model.recommend(2423448)

[9514727, 9667501, 9714376, 9419945, 9761391]

In [5]:
model.predict(2423448,9714376)

0.841934084892273

### Evaluation
Prints the *Precision@K and NDCG@K* metrics for the evaluation of the recommender.

In [6]:
results = model.evaluate_recommender(test_data=test_behavior_df,k=5,n_jobs=4,user_sample=1000)
print("Results")
results

Results


{'Precision@K': np.float64(0.0), 'NDCG@K': np.float64(0.0)}

### Diversity

In [7]:
diversity = model.aggregate_diversity(item_df=article_df,user_sample=1000)
print("Diversity")
diversity

Diversity


0.0002893239463786286

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [8]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(model, "baseline", user_id=2423448, n=5)
footprint

[codecarbon INFO @ 19:49:09] [setup] RAM Tracking...
[codecarbon INFO @ 19:49:09] [setup] CPU Tracking...
[codecarbon WARNING @ 19:49:09] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 19:49:11] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 19:49:11] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 19:49:11] [setup] GPU Tracking...
[codecarbon INFO @ 19:49:11] No GPU found.
[codecarbon INFO @ 19:49:11] >>> Tracker's metadata:
[codecarbon INFO @ 19:49:11]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 19:49:11]   Python version: 3.11.9
[codecarbon INFO @ 19:49:11]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 19:49:11]   Available RAM : 15.731 GB
[codecarbon INFO @ 19:49:11]   CPU count: 20
[codecarbon INFO @ 19:49:11]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 19:49:11]   GPU count: None
[codecarbon INFO @ 19:49:11]   GPU model: None
[codecarbon INFO @ 19:49:14] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\baseline_fit_em

{'fit': (None, 1.138532537674529e-07),
 'recommend': ([9514727, 9667501, 9714376, 9419945, 9761391],
  6.177976078632749e-09)}

### Gini coefficient

In [9]:
gini = model.gini_coefficient(user_sample=1000)
print("Gini Coefficient")
gini

Sampling users
Computing Gini coefficient
[9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 9667501, 9714376, 9419945, 9761391, 9514727, 966

np.float64(0.9970978525827046)